# LangGraph Agent with Human-in-the-Loop

## Setup

In [ ]:
!pip install -U langgraph langchain langchain-core
!pip install --force-reinstall langchain-core
!pip install -U langchain-anthropic
!pip uninstall regex -y
!pip install regex
!pip uninstall tiktoken -y
!pip install tiktoken

In [ ]:
import os
import textwrap
from langchain_core.tools import BaseTool, tool as create_tool
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.types import interrupt, Command

In [ ]:
os.environ["LANGCHAIN_API_KEY"] = "YOUR_LANGCHAIN_API_KEY_HERE"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"

## Define Tool with Interrupt

In [ ]:
def schedule_meeting_with_approval(person: str, date: str, time: str):
   """Schedule a meeting with a person at a specific date and time."""

   response = interrupt(
       f"Schedule meeting with person='{person}', date='{date}', time='{time}'. Approve or edit?"
   )
   
   if response["type"] == "accept":      
       return f" Meeting confirmed with {person} on {date} at {time}."
   elif response["type"] == "edit":
       person = response["args"].get("person", person)
       date = response["args"].get("date", date)
       time = response["args"].get("time", time)
       return f"Meeting confirmed with {person} on {date} at {time}."
   else:
       return "Meeting scheduling cancelled or unknown response."

In [ ]:
checkpointer = InMemorySaver()

agent = create_react_agent(
    model="openai:gpt-4",
    tools=[schedule_meeting_with_approval],

    checkpointer=checkpointer,
)

## Run Agent

In [ ]:
config = {"configurable": {"thread_id": "meeting-123"}}

for chunk in agent.stream(
    {
        "messages": [{
            "role": "user",
            "content": "schedule_meeting with person='Max', date='Tuesday', time='3pm'"
        }]
    },
    config
):
    print(chunk)

In [ ]:
# Getting user input 
person = input("Enter person to meet: ").strip()
date = input("Enter meeting date: ").strip()
time = input("Enter meeting time: ").strip()

# Prepare user message for agent
user_message = f"schedule_meeting with person='{person}', date='{date}', time='{time}'"

interrupt_message = f"""
Here is the proposed meeting schedule:

    Proposed Meeting:
    - With: {person}
    - On: {date}
    - At: {time}

Please approve, edit, reject, or respond with feedback.
"""

print(textwrap.dedent(interrupt_message))

# Human-in-the-loop input
decision = input("Type your decision (accept / edit / reject / respond): ").strip().lower()

if decision == "accept":
    resume_payload = {"type": "accept"}

elif decision == "reject":
    resume_payload = {"type": "reject"}

elif decision == "edit":
    new_person = input(f"New person (leave blank to keep '{person}'): ").strip()
    new_date = input(f"New date (leave blank to keep '{date}'): ").strip()
    new_time = input(f"New time (leave blank to keep '{time}'): ").strip()

    args = {
        "person": new_person if new_person else person,
        "date": new_date if new_date else date,
        "time": new_time if new_time else time,
    }
    resume_payload = {"type": "edit", "args": args}

elif decision == "response":
    feedback = input("Enter your feedback or comment: ").strip()
    resume_payload = {"type": "response", "args": feedback}

else:
    print("Invalid input, rejecting by default.")
    resume_payload = {"type": "reject"}

# Output the choice
print("\nYour choice:", resume_payload)

# Handle decision
if resume_payload["type"] == "accept":
    print(f"\n Meeting confirmed with {person} on {date} at {time}.")

elif resume_payload["type"] == "reject":
    print("\n Meeting scheduling cancelled.")

elif resume_payload["type"] == "edit":
    updated = resume_payload["args"]
    print(f"\n Meeting updated to {updated['person']} on {updated['date']} at {updated['time']}.")

elif resume_payload["type"] == "response":
    print("\n Feedback received:")
    print(f"“{resume_payload['args']}”")